# Inference fine tuned model

In [ ]:
import pandas as pd
df_test = pd.read_csv('test-100-1024.csv')

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Inference API URL
API_URL = "https://demonstem--inference-llada-model-inference.modal.run"

# Create a session to reuse connections
session = requests.Session()

def remote_generate_summary(row_index_prompt):
    idx, prompt = row_index_prompt
    try:
        # print(f"[{idx}] Sending request...")
        response = session.post(API_URL, json={"prompt": prompt})
        response.raise_for_status()
        summary = response.json().get("summary", "")
        # print(f"[{idx}] Received summary.")
        return idx, summary
    except Exception as e:
        print(f"[{idx}] Error: {e}")
        return idx, f"Error: {e}"

# Function to run in parallel
def generate_summaries_parallel(df, max_workers=8):
    results = [None] * len(df)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(remote_generate_summary, (idx, prompt)): idx
            for idx, prompt in enumerate(df["body"])
        }
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx, summary = future.result()
            results[idx] = summary
    return results

# Run inference
df_test['generated'] = generate_summaries_parallel(df_test)

100%|██████████| 100/100 [12:12<00:00,  7.33s/it]


In [ ]:
df_test.to_csv('test-100-1024-generated-fine-tuned-40k-dp.csv', index=False)

# Prep Data for fine tuning

In [ ]:
import pandas as pd
df = pd.read_csv('data/test.csv')
df

,title,body,summary,type,tags,url
0,ย้อนแย้งไหลไปไหลมา พท.สับเละมีชัย,"พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี,ม...",มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ...,"ข่าว,การเมือง","มีชัย ฤชุพันธุ์,สูตรพิสดาร,เลือกตั้ง ส.ส.,พ.ร....",https://www.thairath.co.th/news/politic/1032219
1,ฟลอเรส รับ วัตฟอร์ดห่วยเองเกมพ่ายพาเลซคาบ้าน,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...,NaN,"พรีเมียร์ลีก,วัตฟอร์ด,คริสตัล พาเลซ,กีเก ซานเช...",https://www.thairath.co.th/content/528322
2,เนปาล วิปโยค พายุซัดถล่มเละ,"ตาย30-เจ็บ400 เส้นทางตัดขาด เข้าช่วยลำบาก,เนปา...",เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร...,"ข่าว,ต่างประเทศ","เนปาล,พายุ,ฝนฟ้าคะนอง,ภัยพิบัติจากธรรมชาติ,พาย...",https://www.thairath.co.th/news/foreign/1534976
3,โสมชบาจ๊ะจ๋า 16/05/61,สมเด็จพระเทพรัตนราชสุดาฯ สยามบรมราชกุมารี ทรงเ...,เป็นศูนย์รวมของความคิดสร้างสรรค์และคอมมูนิตี้ค...,NaN,"ODS X GONGKAN,คอลลาโบเรชั่น,อุสรา ยงปิยะกุล,โส...",https://www.thairath.co.th/lifestyle/woman/his...
4,บ้านตะวันลับ ที่พึ่งสุดท้ายของคนชราในเมียนมา,ทุกคนอาจจะต้องการเพียงแค่ใช้ชีวิตในช่วงบั้นปล...,สหประชาชาติประเมินว่า สังคมเมียนมากำลังก้าวเข้...,NaN,NaN,https://thestandard.co/news-world-myanmar-twil...
...,...,...,...,...,...,...
10995,อนาคตใหม่เปิดกลไกติดตาม - มีส่วนร่วมทำงานพรรค-...,อนาคตใหม่ เปิดโครงการ Open FWP Project สร้างกล...,อนาคตใหม่ เปิดโครงการ Open FWP Project สร้างกล...,การเมือง,"แก้รัฐธรรมนูญ,พรรคอนาคตใหม่,ปิยบุตร แสงกนกกุล",https://prachatai.com/journal/2019/07/83650
10996,ศุลกากรไล่ล่าชิปปิ้งโกงภาษี,"ลั่นพบทุจริตจริงถอนใบอนุญาตห้ามรับงานอีก,กรมศุ...",กรมศุลกากร ไล่ล่าชิปปิ้งโกงภาษี สั่งขึ้นบัญชีด...,NaN,"กรมศุลกากร,กุลิศ สมบัติศิริ,ขึ้นบัญชีดำ,แบล็กล...",https://www.thairath.co.th/content/742586
10997,มือปราบเชื้อโรค ศ.นพ.อนุชา อภิสารธนรักษ์ หมอไท...,ถ้าเราทำงานไปโดยไม่ได้คาดหวังอะไร เราจะได้ ต้อ...,นี่คือความลับสวรรค์ที่ คุณหมอเป้อ-ศ.นพ.อนุชา อ...,"ข่าว,สังคม","อนุชา อภิสารธนรักษ์,ทีมข่าวหน้าสตรี,หน่วยโรคติ...",https://www.thairath.co.th/news/society/1417535
10998,นาทีรวบโจรชิงทอง สุดชิล เปลี่ยนชุด ขับเก๋งคันเ...,พบโจรหนุ่มบุกเดี่ยวชิงทอง 70 บาทที่บางบัวทอง เ...,พบโจรหนุ่มบุกเดี่ยวชิงทอง 70 บาทที่บางบัวทอง เ...,"ข่าว,ทั่วไทย","ชิงทอง,โจรชิงทอง,ชิงทอง บางบัวทอง,จับโจรชิงทอง...",https://www.thairath.co.th/news/local/central/...


In [ ]:
# Filter df only with 'body' length higher than 2048
df = df[df['body'].str.len() >= 2048]
len(df)

6069

In [ ]:
# sample 10000 data points from df and save
df.sample(100).to_csv('test-100-geq-2048.csv')

In [ ]:
df = pd.read_csv('test-100-geq-2048.csv')
df['body'].map(len).describe()

,body
count,100.000000
mean,5138.360000
std,4613.804741
min,2051.000000
25%,2770.750000
50%,3711.500000
75%,5255.750000
max,33079.000000


In [ ]:
!wget https://archive.org/download/thaisum_datasets/data.zip

--2025-05-09 13:19:42--  https://archive.org/download/thaisum_datasets/data.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dn790008.ca.archive.org/0/items/thaisum_datasets/data.zip [following]
--2025-05-09 13:19:43--  https://dn790008.ca.archive.org/0/items/thaisum_datasets/data.zip
Resolving dn790008.ca.archive.org (dn790008.ca.archive.org)... 184.105.203.175
Connecting to dn790008.ca.archive.org (dn790008.ca.archive.org)|184.105.203.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 647582078 (618M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 617.58M  18.9MB/s    in 35s     

2025-05-09 13:20:19 (17.7 MB/s) - ‘data.zip’ saved [647582078/647582078]



In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/valid.csv          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._valid.csv  
  inflating: data/test.csv           
  inflating: __MACOSX/data/._test.csv  
  inflating: data/train.csv          
  inflating: __MACOSX/data/._train.csv  


In [ ]:
import pandas as pd
df = pd.read_csv("data/train.csv")

In [ ]:
# Filter df with length body length less than 1024
df = df[df['body'].str.len() < 1024]

In [ ]:
old = pd.read_csv('train-10000-1024.csv')
old2 = pd.read_csv('train-10000-1024-batch-2.csv')
old3 = pd.read_csv('train-10000-1024-batch-3.csv')
old_titles = set(old['title']) | set(old2['title']) | set(old3['title'])
len(old_titles), len(old), len(old2), len(old3)

(29979, 10000, 10000, 10000)

In [ ]:

# Filter df only with 'title' not exist in 'old_titles'
df = df[~df['title'].isin(old_titles)]

In [ ]:
# sample 10000 data points from df and save
df.sample(10000).to_csv('train-10000-1024-batch-4.csv')

In [ ]:
len(df)

12796

# Langchain Techniques


In [ ]:
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00


In [ ]:
%pip install langchain


In [ ]:
from typing import Any, Dict, Iterator, List, Mapping, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk

import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class CustomLLM(LLM):
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
      API_URL = "https://demonstem--inference-llada-model-inference.modal.run"

      # Create a session to reuse connections
      session = requests.Session()
      # try:
      response = session.post(API_URL, json={"prompt": prompt})
      response.raise_for_status()
      summary = response.json().get("summary", "")
      return summary
      # except Exception as e:
      #     print(f"[{idx}] Error: {e}")
      #     return idx, f"Error: {e}"

    def _stream(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[GenerationChunk]:
        """Stream the LLM on the given prompt.

        This method should be overridden by subclasses that support streaming.

        If not implemented, the default behavior of calls to stream will be to
        fallback to the non-streaming version of the model and return
        the output as a single chunk.

        Args:
            prompt: The prompt to generate from.
            stop: Stop words to use when generating. Model output is cut off at the
                first occurrence of any of these substrings.
            run_manager: Callback manager for the run.
            **kwargs: Arbitrary additional keyword arguments. These are usually passed
                to the model provider API call.

        Returns:
            An iterator of GenerationChunks.
        """
        for char in prompt[: self.n]:
            chunk = GenerationChunk(text=char)
            if run_manager:
                run_manager.on_llm_new_token(chunk.text, chunk=chunk)

            yield chunk

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "custom"

In [ ]:
llm = CustomLLM()

In [ ]:
llm.invoke("เว็บไซต์ต่างประเทศ รายงานเหตุการณ์ระทึก เมื่อแก๊งอินฟลูกว่า 32 ชีวิต รวมตัวจัดปาร์ตี้บนเรือยอร์ชหรูราคา 147 ล้านบาท สุดท้ายเรือค่อยๆ จมลงกลางทะเลใกล้เกาะไมอามี บีช รัฐฟลอริดา สหรัฐ เจ้าหน้าที่รีบเข้าช่วยเหลือ อพยพคนบนเรือได้สำเร็จ ไม่มีรายงานผู้ได้รับบาดเจ็บ")

'เว็บไซต์ต่างประเทศ รายงานเหตุการณ์ระทึก เมื่อแก๊งอินฟลูกว่า 32 ชีวิต รวมตัวจัดปาร์ตี้บนเรือยอร์ชหรูราคา 147 ล้านบาท สุดท้ายเรือค่อยๆ จมลงกลางทะเลใกล้เกาะไมอามี บีช รัฐฟลอริดา สหรัฐ เจ้าหน้าที่รีบเข้าช่วยเหลือ อพยพคนบนเรือได้สำเร็จ'

## Map-Reduce

In [ ]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_core.documents import Document

# Map
map_template = """สรุปข้อความต่อไปนี้:
{docs}"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """สรุปข้อความต่อไปนี้:
{docs}"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
  llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=1024,
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


raw_text = """สมเด็จพระสันตะปาปาฟรานซิส[1][ก] (อังกฤษ: Francis; ละติน: Franciscus; อิตาลี: Francesco; สเปน: Francisco; พระนามเดิม ฮอร์เฮ มาริโอ เบร์โกกลิโอ[a][ข]; 17 ธันวาคม ค.ศ. 1936 – 21 เมษายน ค.ศ. 2025) เป็นอดีตประมุขแห่งพระศาสนจักรคาทอลิก และอดีตประมุขแห่งนครรัฐวาติกันตั้งแต่วันที่ 13 มีนาคม ค.ศ. 2013 จนกระทั่งสิ้นพระชนม์ สมเด็จพระสันตะปาปาฟรานซิสเป็นสมเด็จพระสันตะปาปาพระองค์แรกจากคณะเยสุอิต เป็นพระองค์แรกที่มาจากทวีปอเมริกาใต้ เป็นพระองค์แรกที่มาจากซีกโลกใต้ และเป็นพระองค์แรกที่มาจากนอกทวีปยุโรป ตั้งแต่สมัยของสมเด็จพระสันตะปาปาเกรกอรีที่ 3 ซึ่งเป็นชาวซีเรีย ผู้ซึ่งดำรงตำแหน่งบิชอบแห่งกรุงโรมในคริสต์ศตวรรษที่ 8

สมเด็จพระสันตะปาปาฟรานซิสทรงแต่งตั้งให้สตรีเป็นสมาชิกเต็มของสมณสภาปกครองโรมัน[2][3] พระองค์ทรงยืนกรานว่าคริสตจักรนิกายโรมันคาทอลิกควรเห็นอกเห็นใจสมาชิกกลุ่มบุคคลที่มีความหลากหลายทางเพศมากกว่านี้ และทรงระบุว่าแม้ว่าจะไม่อนุญาตให้มีการให้พรแก่การสมรสเพศเดียวกัน แต่บุคคลต่าง ๆ ก็สามารถได้รับพรได้ตราบใดที่ไม่มีการให้พรในบริบทของพิธีกรรม[4] สมเด็จพระสันตะปาปาฟรานซิสทรงเป็นผู้วิจารณ์ระบบทุนนิยม บริโภคนิยมที่ไร้ขอบเขต และการพัฒนาที่มากเกินไป[5] พระองค์ทรงทำให้การดำเนินการเกี่ยวกับการเปลี่ยนแปลงสภาพภูมิอากาศเป็นจุดเน้นหลักของตำแหน่งพระสันตปาปาของพระองค์[6] พระองค์ได้รับการตีความอย่างกว้างขวาง ทรงประณามโทษประหารชีวิตว่าเป็นความชั่วร้ายในตัวเอง[7] โดยทรงระบุว่าคริสตจักรนิกายโรมันคาทอลิกมุ่งมั่นที่จะยกเลิกโทษประหารชีวิต[8] ในการเจรจาระหว่างประเทศ สมเด็จพระสันตะปาฟรานซิสทรงวิพากษ์วิจารณ์การเพิ่มขึ้นของประชานิยมปีกขวา เรียกร้องให้ยกเลิกการก่ออาชญากรรมต่อคนรักร่วมเพศ[9] ช่วยฟื้นฟูความสัมพันธ์ทางการทูตอย่างเต็มรูปแบบระหว่างสหรัฐอเมริกาและประเทศคิวบา เจรจาข้อตกลงกับจีนเพื่อกำหนดว่าพรรคคอมมิวนิสต์มีอิทธิพลมากเพียงใดในการแต่งตั้งบิชอปชาวจีน และสนับสนุนประเด็นผู้ลี้ภัย พระองค์เรียกร้องให้การคุ้มครองผู้อพยพเป็น "หน้าที่ของอารยธรรม" และวิพากษ์วิจารณ์การเมืองต่อต้านผู้อพยพ รวมถึงนโยบายของประธานาธิบดีดอนัลด์ ทรัมป์แห่งสหรัฐอเมริกา[10][11] ใน ค.ศ. 2022 พระองค์ทรงขออภัยต่อบทบาทของคริสตจักรใน "การฆ่าล้างเผ่าพันธุ์ทางวัฒนธรรม" ของชนพื้นเมืองในแคนาดา[12] สมเด็จพระสันตะปาปาฟรานซิสทรงเรียกประชุมสมัชชาซีนอดซึ่งได้รับการอธิบายว่าเป็นจุดสูงสุดของการดำรงตำแหน่งพระสันตปาปาของพระองค์และเป็นเหตุการณ์ที่สำคัญที่สุดในคริสตจักรนิกายโรมันคาทอลิกตั้งแต่สังคายนาวาติกันครั้งที่สอง
"""
docs = [Document(page_content=raw_text, metadata={"source": "example.txt"})]
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

<ipython-input-33-847931e4e071>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  map_chain = LLMChain(llm=llm, prompt=map_prompt)
<ipython-input-33-847931e4e071>:21: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(
<ipython-input-33-847931e4e071>:26: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended replacement: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain/
  reduce_documents_chain = ReduceDocumentsChain(
<ipython-input-33-847931e4e071>:36: LangChainDeprecationWarning: This class is deprecated. Please see the migration guide here for a recommended repla

In [ ]:
map_reduce_chain.run(split_docs)

'สมเด็จพระสันตะปาปาฟรานซิสเป็นอดีตประมุขแห่งพระศาสนจักรคาทอลิก และอดีตประมุขแห่งนครรัฐวาติกันตั้งแต่วันที่ 13 มีนาคม ค.ศ. 2013 จนกระทั่งสิ้นพระชนม์ สมเด็จพระสันตะปาปาฟรานซิสเป็นสมเด็จพระสันตะปาปาพระองค์แรกจากคณะเยสุอิต'

In [ ]:
from langchain.chains.summarize import load_summarize_chain
prompt_template = """สรุปข้อความต่อไปนี้:
{text}"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = """สรุปข้อความต่อไปนี้:
{existing_answer}
------------
{text}
------------"""
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

In [ ]:
result = chain({"input_documents": split_docs}, return_only_outputs=True)

<ipython-input-20-95f9558ba101>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"input_documents": split_docs}, return_only_outputs=True)


In [ ]:
result["output_text"]

'สมเด็จพระสันตะปาปาฟรานซิส เป็นอดีตประมุขแห่งพระศาสนจักรคาทอลิก และอดีตประมุขแห่งนครรัฐวาติกันตั้งแต่วันที่ 13 มีนาคม ค.ศ. 2013 จนกระทั่งสิ้นพระชนม์ สมเด็จพระสันตะปาปาฟรานซิสเป็นสมเด็จพระสันตะปาปาพระองค์แรกจากคณะเยสุอิต และจากสมณสภาปกครองโรมัน'

In [ ]:
import pandas as pd

df = pd.read_csv('test-100-geq-2048.csv')

Inference Refine

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# Define the function to process a single row
def process_row(i, df, map_reduce_chain):
    raw_text = df['body'][i]
    docs = [Document(page_content=raw_text, metadata={"source": "example.txt"})]
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    split_docs = text_splitter.split_documents(docs)
    return chain({"input_documents": split_docs}, return_only_outputs=True)["output_text"]

# Main code
# df = df.iloc[:2]
results = []
# Set max_workers to control the number of threads (e.g., 4)
with ThreadPoolExecutor(max_workers=8) as executor:
    # Map the process_row function to all indices with tqdm progress bar
    results = list(tqdm(
        executor.map(lambda i: process_row(i, df, map_reduce_chain), range(len(df))),
        total=len(df)
    ))

<ipython-input-35-2d7ad92ef974>:14: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain({"input_documents": split_docs}, return_only_outputs=True)["output_text"]
  0%|          | 0/100 [00:27<?, ?it/s]


HTTPError: 500 Server Error: Internal Server Error for url: https://nacnano--inference-llada-model-inference.modal.run/

In [ ]:
# Create df and save csv
df = pd.DataFrame({'body': df['body'], 'generated': results})
df.to_csv('langchain-refine-40k-dp.csv')

Inference Map-Reduce

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# Define the function to process a single row
def process_row(i, df, map_reduce_chain):
    raw_text = df['body'][i]
    docs = [Document(page_content=raw_text, metadata={"source": "example.txt"})]
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000, chunk_overlap=0
    )
    split_docs = text_splitter.split_documents(docs)
    return map_reduce_chain.run(split_docs)

# Main code
# df = df.iloc[:2]
results = []
# Set max_workers to control the number of threads (e.g., 4)
with ThreadPoolExecutor(max_workers=8) as executor:
    # Map the process_row function to all indices with tqdm progress bar
    results = list(tqdm(
        executor.map(lambda i: process_row(i, df, map_reduce_chain), range(len(df))),
        total=len(df)
    ))

<ipython-input-15-0fd3949bd2ae>:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return map_reduce_chain.run(split_docs)
  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

100%|██████████| 100/100 [14:59<00:00,  8.99s/it]


In [ ]:
# Create df and save csv
df = pd.DataFrame({'body': df['body'], 'generated': results})
df.to_csv('langchain-map-reduce-40k-dp.csv')

# Typhoon Inference

In [ ]:
!pip install --upgrade together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
from together import Together
from google.colab import userdata

client = Together(api_key=userdata.get("TOGETHER_AI_API_KEY"))

response = client.chat.completions.create(
    model="scb10x/scb10x-llama3-1-typhoon2-8b-instruct",
    messages=[{"role": "user", "content": "What are some fun things to do in New York?"}]
)
print(response.choices[0].message.content)

Some fun things to do in New York include visiting the Statue of Liberty, exploring Central Park, taking a stroll through Times Square, visiting the Metropolitan Museum of Art, and enjoying a Broadway show.


In [ ]:
import pandas as pd
df = pd.read_csv('test-100-1024.csv')

,title,body,summary,type,tags,url
0,เชียงใหม่เร่งกลบข่าวลือพท.ก่อการร้าย,ศูนย์ข่าวภาคเหนือ -12 เม.ย.2548ตำรวจท่องเที่ยว...,ศูนย์ข่าวภาคเหนือ -12 เม.ย.2548 ตำรวจท่องเที่ย...,สิ่งแวดล้อม,NaN,https://prachatai.com/journal/2005/04/3625
1,โควิดสุดร้ายกาจ ระบาด 3 เดือน ติดเชื้อทั่วโลก...,ไม่น่าเชื่อ เชื้อโควิด 19 ระบาดใหญ่ทั่วโลก ผ่า...,ไม่น่าเชื่อ เชื้อโควิด-19 ระบาดใหญ่ทั่วโลก ผ่า...,"ข่าว,ต่างประเทศ","โควิด-19,ไวรัสโคโรน่า,ไวรัสโคโรนา,ยอดโควิดทั่ว...",https://www.thairath.co.th/news/foreign/1820601
2,มจร.จัดพิธีประสาทปริญญาบัตรปี 61,"http://www.mcu.ac.th/,เมื่อวันที่ 10 พ.ค. ที่อ...",ที่อาคาร มวก. 48 พรรษา มหาวิทยาลัยมหาจุฬาลงกรณ...,"ข่าว,ทั่วไทย","พระพรหมบัณฑิต,มหาวิทยาลัยมหาจุฬาลงกรณราชวิทยาล...",https://www.thairath.co.th/news/local/1278441
3,ทวิตเตอร์ เฟซบุ๊ก แจกเงินพนักงานทำงานที่บ้านคน...,บริษัทเทคโนโลยีหลายแห่งในสหรัฐอเมริกา ทวิตเตอร...,บริษัทเทคโนโลยีหลายแห่งในสหรัฐอเมริกา ทวิตเตอร...,"ข่าว,ไอที","แจกเงินพนักงาน,ทวิตเตอร์,เฟซบุ๊ก,ทำงานที่บ้าน,...",https://www.thairath.co.th/news/tech/1843934
4,ด่วน วัยรุ่น 3 สาว ชวนกันมากระโดดสะพานพระราม 8...,3 สาววัยรุ่นชวนกันมากระโดดแม่น้ำเจ้าพระยา โดยก...,3 สาววัยรุ่น ชวนกันมากระโดดแม่น้ำเจ้าพระยา โดย...,"ข่าว,อาชญากรรม","กระโดดน้ำตาย,3สาวกระโดดน้ำตาย,กระโดดสะพานพระรา...",https://www.thairath.co.th/news/crime/1812296


In [ ]:
body = []
generated = []

In [ ]:
import time
from tqdm import tqdm

for i in tqdm(range(len(df))):
  response = client.chat.completions.create(
      model="scb10x/scb10x-llama3-1-typhoon2-8b-instruct",
      messages=[{"role": "user", "content": "สรุปข้อความต่อไปนี้\n"+df['body'][i]}]
  )
  body.append(df['body'][i])
  generated.append(response.choices[0].message.content)
  time.sleep(1)

100%|██████████| 100/100 [07:43<00:00,  4.64s/it]


In [ ]:
# Create dataframe from body, generated
df = pd.DataFrame({'body': body, 'generated': generated})
df.to_csv('typhoon_inference.csv')

In [ ]:
df2 = pd.read_csv('gemini_summaries.csv')

In [ ]:
df2.iloc[60]['body'], df2.iloc[60]['generated']

('เพิ่งสะบั้นรักกันไปหยกๆ มาสัปดาห์นี้ในละคร ร้อยป่า นางเอกสาว ปูเป้-เกศรินทร์ หายโกรธแล้ว แถมยังเป็นฝ่ายไปจูบพระเอก บิ๊กเอ็ม-กฤตฤทธิ์ ก่อนอีกตะหากในฉาก ที่ เสือ กลิ่นสัก (บิ๊กเอ็ม) เอ่ยปากขอ คุณแป๋ว (ปูเป้) แต่งงานและยังขอจูบมัดจำ 1 จุ๊บก่อน แต่ คุณแป๋ว ไม่ให้ เสือ ก็เลยมีงอน คุณแป๋ว เลยต้องจูจุ๊บ เสือ เพื่อง้อเลยโดน เสือ จับจูบแบบดูดดื่มซะเลย>> อ่านเรื่องย่อนิยายทุกเรื่อง คลิกที่นี่ <<ผกก.อ้างว่าเป็นฉากสำคัญเลยขอจูบจริง บิ๊กเอ็ม ไม่มีปัญหา ส่วน ปูเป้ หน้าแดงพูดด้วยความเขิน ก็ได้ค่ะ แต่พี่บิ๊กเอ็มห้ามแกล้งหนู บิ๊กเอ็ม หัวเราะก่อนตอบตกลง ผกก.ให้ซักซ้อมเบาๆเพื่อได้ความสดตอนถ่ายจริงเมื่อเริ่มถ่าย บิ๊กเอ็ม พูดขอแต่งงานก่อน พอถึงตอนขอจูบมัดจำ ปูเป้ เซย์โน บิ๊กเอ็ม ก็ทำงอน ปูเป้ เลยจูบง้อไป 1 จุ๊บเบาๆ บิ๊กเอ็ม ได้ทีคว้า ปูเป้ หมับจับจูบประกบปากแบบเต็มๆ แบบนี้เขาถึงว่าจูบจ้ะ จูบสอนเด็กมันซะเลย (ฮา)อย่าพลาดชม วันศุกร์-อาทิตย์ นี้ 20.30 น. ทางช่อง 7 HD.',
 'ในละคร "ร้อยป่า" นางเอก "ปูเป้" จูบง้อพระเอก "บิ๊กเอ็ม" ที่งอนหลังขอแต่งงานและจูบมัดจำไม่สำเร็จ ผู้กำกับสั่งจูบจริงในฉากสำคัญ "บิ๊กเอ็ม" ไ

In [ ]:
df2.iloc[61]['body'], df2.iloc[61]['generated']

('11 มีนาคม 2553 สถานทูตสหรัฐอเมริกาประจำประเทศไทย ออกแถลงการณ์เกี่ยวกับสถานการณ์การเมืองไทยปัจจุบัน รวมถึงการชุมนุมที่จะมีขึ้นช่วงสุดสัปดาห์นี้โดยในแถลงการณ์ระบุว่า ประเทศไทยควรแก้ไขปัญหาความเห็นที่แตกต่างกันโดยสถาบันประชาธิปไตยและไม่ใช้ความรุนแรง ทั้งยังเรียกร้องให้ฝ่ายผู้ชุมนุมสาบานว่า จะไม่กระทำรุนแรง และชุมนุมอยู่ภายใต้กฏหมาย ขณะเดียวกันก็บอกให้รัฐบาลไทยอดทนอดกลั้นต่อผู้ประท้วงประเทศไทยถือว่าเป็นเพื่อนที่ใกล้ชิดและเป็นพันธมิตรกับสหรัฐอเมริกาและในฐานะที่เราเป็นมิตรกันมายาวนาน พวกเราก็เฝ้ามองสถานการณ์ปัจจุบันในไทยอย่างใกล้ชิด ทางสหรัฐฯ เชื่อว่าปัญหาความแตกต่างทางความคิดควรได้รับการแก้ไขผ่านทางสถาบันที่เป็นประชาธิปไตยของไทย และไม่ใช่แก้ไขโดยการใช้กำลังการชุมนุมอย่างสงบเป็นเครื่องหมายของสังคมประชาธิปไตย พวกเราเรียกร้องให้ทั้งกลุ่มผู้ชุมนุมและแกนนำให้คำมั่นว่าจะไม่ใช้ความรุนแรง เพื่อแสดงการใช้สิทธิในการชุมนุมประท้วงอย่างสันติและอยู่ภายใต้กฏหมาย ทั้งนี้พวกเรายังได้สนับสนุนให้รัฐบาลไทยมีความอดทนอดกลั้นต่อผู้ประท้วงอย่างเหมาะสม',
 'สถานทูตสหรัฐฯ แถลงการณ์ 11 มี.ค. 2553 เรียกร้องทุกฝ่ายในไ